# **ReelFeel - IMDB Reviews Sentiment Classification Model with Natural Language Processing - using Recurrent Neural Networks**

---

**Alam Rincon - [GitHub: MrRincon](https://github.com/MrRincon)**

**Petar Atanasov - [GitHub: petar-Atanasov](https://github.com/petar-Atanasov)**

**Teon Morgan - [GitHub: Mi1kDev](https://github.com/Mi1kDev)**

---

**Lakshmipathi N. (2019) ‘IMDB Dataset of 50K Movie Reviews’. Available at: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews (Accessed: 14 April 2025).**

# **Preinstalling Libraries**

Run once and restart the kernel. Do not run again, and continue.

In [1]:
!pip install gensim

# **Preprocessing Data**

Importing core python libraries
*   pandas for dataset manipulation
*   numpy for mathematical processes
*   pyplot and seaborn for data visualization

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

nlp_dataset = pd.read_csv("./datasets/IMDB Dataset.csv")
nlp_dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Display general information breakdown of the dataset

In [2]:
# describes basic information regarding the dataset
nlp_dataset.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [3]:
# indicates datatyes of the various data columns
nlp_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


# **Deep Learning Model Implementation**

## **Recurrent Neural Network (RNN)**

## **Model Training**

# **Evaluation and Insights**